# Genetic Algorithm for solving the p-median problem from the Shell AI Hackathon

In [3]:
from pymoo.optimize import minimize

1. Problem
2. Algorithm
3. Stop criteria

## 1. Problem Definition

The **capacitated p-median problem** is a well-known optimization problem in operations research. It involves selecting *p* facilities from a set of potential locations to minimize the total distance between the facilities and the demand points (or customers) they serve. The problem is often used in facility location planning, where the goal is to determine the optimal location of facilities such as warehouses, hospitals, or schools to serve a given population. 

The problem is NP-hard, meaning that it is computationally infeasible to solve for large instances using exact methods. Therefore, heuristic and metaheuristic algorithms such as genetic algorithms are often used to find approximate solutions.

![Alt text](image.png)

#### Example:

*Medians* $= \{1,4\}$

Vertex | 1 | 2 | 3 | 4 | 5 | $\dots$ | n-1 | n
--- | --- | --- | --- |--- |--- |--- |--- |---
Median | -1 | 4 | 5 | -1 | 1 | $\dots$ | $m_{n-1}$|$m_n$



In [ ]:
from pymoo.model.problem import Problem
import numpy as np

def create_problem_data(num_harvesting_sites, num_warehouses, B, D, C, a=0.001, b=1):

    # create the objective function
    cost_of_transportation = np.sum(x * B.reshape(-1, 1) * D)
    cost_of_underutilization = np.sum(C - np.sum(x, axis=0))
    objective_function = a * cost_of_transportation + b * cost_of_underutilization

    return objective_function


class ProblemWrapper(Problem):
    def _evaluate(self, X, out, *args, **kwargs):
        